### Import modules and initialize

In [2]:
import os
import ee
import folium
import shapefile
import numpy as np

In [3]:
ee.Initialize()

### Inputs

In [23]:
TILES_DIR = '../../data/ghana/tiles'
SHP_DIR_LIST = os.listdir(TILES_DIR)
shp_list = []
for SHP_DIR in SHP_DIR_LIST:
    shp = os.path.join(TILES_DIR, SHP_DIR, SHP_DIR+'.shp')
    shp_list.append(shp)

In [34]:
for shp_fp in shp_list:
    print(os.path.basename(shp_fp)[:-4])

ghana_l_23
ghana_s_12
ghana_l_14
ghana_l_21
ghana_l_0
ghana_l_11
ghana_s_0
ghana_s_20
ghana_l_1
ghana_s_13
ghana_s_4
ghana_l_22
ghana_l_2
ghana_l_19
ghana_l_5
ghana_l_10
ghana_s_10
ghana_s_5
ghana_s_9
ghana_s_8
ghana_l_4
ghana_l_9
ghana_s_7
ghana_s_15
ghana_s_19
ghana_l_17
ghana_l_7
ghana_l_12
ghana_l_8
ghana_s_2
ghana_l_13
ghana_l_6
ghana_s_18
ghana_s_11
ghana_s_3
ghana_l_18
ghana_s_16
ghana_l_15
ghana_s_1
ghana_s_6
ghana_s_23
ghana_s_17
ghana_s_22
ghana_l_3
ghana_l_16
ghana_s_21
ghana_s_14
ghana_l_20


### Collect imagery

In [35]:
# function to convert shapefile into feature collection
def convert_shp2feature (shp_fp):
    shp = shapefile.Reader(shp_fp)
    fields = shp.fields[1:]
    field_names = [field[0] for field in fields]
    wgs84 = ee.Projection('EPSG:4326')
    features = []
    for sr in shp.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        ee_geometry = ee.Geometry(geom,'EPSG:4326')
        feat = ee.Feature(ee_geometry, atr)
        features.append(feat)
    fc = ee.FeatureCollection(features)
    feature = ee.Feature(fc.first())
    feature.set('tile', os.path.basename(shp_fp)[:-4])
    return feature

# function to cloud mask sentinel imagery
def mask_clouds_sentinel (image):
    qa = image.select('QA60')
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)
    clear_pixels = qa.bitwiseAnd(cloudBitMask).eq(0)\
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
        
    return image.updateMask(clear_pixels).divide(10000)

# function to calculate centroid of feature
def calc_centroid (feature):
    # obtain geometry
    bounds = feature.geometry()
    
    # calculate centroid
    xmax = bounds.coordinates().getInfo()[0][2][0]
    xmin = bounds.coordinates().getInfo()[0][0][0]
    ymin = bounds.coordinates().getInfo()[0][3][1]
    ymax = bounds.coordinates().getInfo()[0][1][1]
    xmean = np.average([xmin, xmax])
    ymean = np.average([ymin, ymax])
    
    return [ymean, xmean], ee.Geometry.Point([xmean, ymean]), [[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]]

In [36]:
ee_bounds_list = []

for shp in shp_list:
    ee_bounds_list.append(convert_shp2feature(shp))

In [20]:
poi_list = []
centroid_list = []
coord_bounds_list = []

for ee_bounds in ee_bounds_list:
    poi, centroid, coord_bounds = calc_centroid(ee_bounds)
    poi_list.append(poi)
    centroid_list.append(centroid)
    coord_bounds_list.append(coord_bounds)

In [21]:
median_image_list = []
sentinel_sr_collection = image = ee.ImageCollection('COPERNICUS/S2_SR')

for i in range(len(centroid_list)):
    median_image = sentinel_sr_collection\
        .filterBounds(centroid[i])\
        .map(mask_clouds_sentinel)\
        .median()\
        .clip(ee_bounds[i])
    median_image_list.append(median_image)

### Display

Skip if not testing

In [7]:
# function to display ee image tiles on folium map
def add_ee_layer(self, eeImageObject, visParams, name):
  mapID = ee.Image(eeImageObject).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+"/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

In [8]:
# create map
folium.Map.add_ee_layer = add_ee_layer
eeMap = folium.Map(location=poi, zoom_start=8)

In [9]:
# define visualization parameters
sentinel_sr_vis_params = {
    'bands':['B4','B3','B2'], 
    'min':0, 
    'max':0.2
}

In [10]:
# add layers to map
eeMap.add_ee_layer(image, sentinel_sr_vis_params, 'precipitation')

In [11]:
# display map
eeMap.add_child(folium.LayerControl())
folium.Marker(poi).add_to(eeMap)
display(eeMap)

### Save to Google Drive

In [24]:
median_image_list

In [12]:


coord_bounds = ee_bounds.geometry().coordinates().getInfo()[0][:4]

task_config = {
    'folder': 'ghana',
    'scale': 10,
    'region': coord_bounds,
    'maxPixels': 10000000000
}

image = image.clip(ee_bounds)
task = ee.batch.Export.image(image, 'ghana_test_01', task_config)

In [13]:
task.start()